In [ ]:
import torch
import torch.nn as nn
import torch.autograd as autograd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Use GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
torch.manual_seed(42)
np.random.seed(42)

In [ ]:
# Normalize alpha values
def normalise_alpha(alpha):
    """
    Map the physical parameter alpha ∈ [1,10] to a normalized value in [0,1].

    Reason:
      - Neural networks usually train better if all inputs are on a similar scale
        (e.g., ~[0,1] or [-1,1]).
      - Here, alpha is originally in [1,10], so we linearly rescale it.

    Formula:
      alpha_norm = (alpha - 1) / 9
      so that alpha = 1 -> 0, alpha = 10 -> 1.
    """
    return (alpha - 1.) / 9.

# Denormalize alpha values
def denormalise_alpha(alpha_tilde):
    """
    Inverse mapping: convert normalized alpha_tilde ∈ [0,1] back to physical alpha ∈ [1,10].

    Formula:
      alpha = alpha_tilde * 9 + 1
      so that alpha_tilde = 0 -> 1, alpha_tilde = 1 -> 10.
    """
    return alpha_tilde * 9. + 1.


# Collocation points --------------------------------------------------------
# Number of interior (x, alpha) points where we enforce the PDE residual.
n_collocation = 10_000

# Sample x uniformly in the spatial domain [0, 1].
# Shape: (n_collocation, 1)
x_colloc = torch.rand(n_collocation, 1, device=device)

# Sample alpha uniformly in the physical range [1, 10].
# Shape: (n_collocation, 1)
alpha_colloc = 1 + 9 * torch.rand(n_collocation, 1, device=device)  # physical alpha

# Normalized alpha used as NN input (better numerical behavior).
alpha_colloc_norm = normalise_alpha(alpha_colloc)


# Boundary points -----------------------------------------------------------
# Number of boundary points used to enforce u(0, alpha) = 0 and u(1, alpha) = 0.
n_boundary = 1000

# Sample alpha from the same distribution [1, 10] for boundary points
# so that boundary conditions are enforced across the full parameter range.
alpha_bc = 1 + 9 * torch.rand(n_boundary, 1, device=device)
alpha_bc_norm = normalise_alpha(alpha_bc)

# Left boundary: x = 0, for all sampled alpha_bc.
x_bc_left  = torch.zeros_like(alpha_bc, device=device)  # shape (n_boundary, 1)

# Right boundary: x = 1, for all sampled alpha_bc.
x_bc_right = torch.ones_like(alpha_bc, device=device)   # shape (n_boundary, 1)


In [ ]:

# Parametric PINN model -----------------------------------------------------
class ParametricPINN(nn.Module):
    def __init__(self, layers=(2, 64, 64, 64, 64, 1)):
        """
        layers: tuple specifying network architecture:
          (input_dim, hidden1, hidden2, ..., hiddenK, output_dim)

        Here:
          (2, 64, 64, 64, 64, 1) means:
            - input_dim = 2  (x, alpha_norm)
            - 4 hidden layers with 64 neurons each
            - output_dim = 1  (scalar u(x; alpha))
        """
        super().__init__()

        net_layers = []
        # Build all hidden layers with Tanh activation
        # layers[:-2] = all except last two: (2, 64, 64, 64)
        # layers[1:-1] = from second to second-last: (64, 64, 64, 64)
        # zip(...) gives pairs: (2->64), (64->64), (64->64), (64->64)
        for in_dim, out_dim in zip(layers[:-2], layers[1:-1]):
            net_layers.append(nn.Linear(in_dim, out_dim))
            net_layers.append(nn.Tanh())

        # Final layer: maps last hidden size -> output_dim (no activation)
        net_layers.append(nn.Linear(layers[-2], layers[-1]))

        # Wrap into a Sequential module for easy forward pass.
        self.network = nn.Sequential(*net_layers)

    def forward(self, x, alpha_norm):
        """
        Forward pass of the parametric PINN.

        Inputs:
          x          : tensor of shape (N, 1), spatial coordinates in [0,1]
          alpha_norm : tensor of shape (N, 1), normalized alpha in [0,1]

        We concatenate x and alpha_norm along the feature dimension so that
        each input sample is [x_i, alpha_norm_i].

        Output:
          u(x, alpha_norm): shape (N, 1), approximate solution of PDE.
        """
        # Concatenate spatial and parameter inputs -> shape (N, 2)
        inputs = torch.cat([x, alpha_norm], dim=1)
        return self.network(inputs)


# Instantiate the PINN and move it to the desired device (CPU/GPU).
pinn = ParametricPINN().to(device)


# PDE residual computation --------------------------------------------------
def pde_residual(model, x, alpha, alpha_norm):
    """
    Compute the residual of the PDE:

        u_xx(x; alpha) + alpha = 0

    We return:
        r(x, alpha) = u_xx(x; alpha) + alpha

    so that the PINN will try to make r ≈ 0 at collocation points.

    Inputs:
      model      : ParametricPINN instance
      x          : (N,1) tensor of spatial coordinates
      alpha      : (N,1) tensor of *physical* alpha values
      alpha_norm : (N,1) tensor of *normalized* alpha values (used as NN input)

    Note:
      - We do NOT differentiate w.r.t. alpha here, only w.r.t. x.
      - alpha is treated as constant in the PDE (parameter, not variable).
    """
    # Enable gradients w.r.t. x so autograd can compute u_x and u_xx.
    x.requires_grad_(True)
    # Alpha_norm is not used for derivatives, so we keep requires_grad = False.
    alpha_norm.requires_grad_(False)

    # Forward pass: u(x; alpha_norm)
    u = model(x, alpha_norm)  # shape (N,1)

    # First derivative du/dx.
    #   grad_outputs=torch.ones_like(u) selects elementwise derivatives.
    u_x = autograd.grad(
        u, x,
        grad_outputs=torch.ones_like(u),
        create_graph=True)[0]

    # Second derivative d^2u/dx^2 by differentiating u_x w.r.t x again.
    u_xx = autograd.grad(
        u_x, x,
        grad_outputs=torch.ones_like(u_x),
        create_graph=True)[0]

    # PDE residual: u_xx + alpha should be zero ideally.
    # -d^2u/dx^2 = alpha ~> d^2u/dx^2 + alpha = 0
    return u_xx + alpha  # residual ~ 0 at collocation points



# Optimizer and training ----------------------------------------------------
optimizer = torch.optim.Adam(pinn.parameters(), lr=1e-3)

# Weight for pde term in total loss.
W_pde = 1.0

# Weight for boundary condition term in total loss.
W_bc = 1.0


def train(num_iterations=10_000, print_every=1000):
    """
    Train the parametric PINN using a simple gradient-descent loop.

    Loss = PDE residual loss + W_bc * boundary condition loss

    PDE residual loss:
      - Mean-squared error of r(x, alpha) = u_xx + alpha at collocation points.

    Boundary condition loss:
      - Enforce u(0, alpha) = 0 and u(1, alpha) = 0 for randomly sampled alphas.
    """
    for it in range(1, num_iterations + 1):
        optimizer.zero_grad()

        # ---------------------------
        # PDE residual loss (interior)
        # ---------------------------
        res = pde_residual(pinn, x_colloc, alpha_colloc, alpha_colloc_norm)
        loss_pde = torch.mean(res**2)

        # ---------------------------
        # Boundary condition loss
        #   BC: u(0; alpha) = 0, u(1; alpha) = 0
        # ---------------------------
        u_left  = pinn(x_bc_left,  alpha_bc_norm)  # u(0; alpha)
        u_right = pinn(x_bc_right, alpha_bc_norm)  # u(1; alpha)

        # Penalize deviations from zero at both boundaries.
        loss_bc = torch.mean(u_left**2) + torch.mean(u_right**2)

        # Total loss combines PDE and BC terms.
        loss = W_pde * loss_pde + W_bc * loss_bc

        # Backpropagation: compute d(loss)/d(parameters)
        loss.backward()

        # Update network parameters with Adam optimizer
        optimizer.step()

        # Console logging
        if it % print_every == 0 or it == 1:
            print(
                f"Iter {it:6d} | Loss: {loss.item():.4e} "
                f"| PDE: {loss_pde.item():.2e} | BC: {loss_bc.item():.2e}"
            )


# Uncomment the next line to train (takes ~1–2 min on CPU)
train()


# True analytical solution --------------------------------------------------
def true_solution(x, alpha):
    """
    Closed-form solution of the boundary value problem:

        u_xx + alpha = 0,   x ∈ [0,1]
        u(0) = 0, u(1) = 0

    Solve ODE:
      u''(x) = -alpha
      Integrate twice:
         u(x) = -alpha * x^2 / 2 + C1 * x + C2

      Apply boundary conditions:
         u(0) = 0 → C2 = 0
         u(1) = 0 → -alpha/2 + C1 = 0 → C1 = alpha/2

      So:
         u(x) = (alpha/2) * x * (1 - x)
    """
    return (alpha / 2.0) * x * (1 - x)


# Visualization -------------------------------------------------------------
def plot_predictions(model, alphas=[1, 3, 5, 7, 10], num_points=200):
    """
    Plot PINN predictions vs. analytical solution for several alpha values.

    Inputs:
      model     : trained ParametricPINN
      alphas    : list of physical alpha values to evaluate
      num_points: number of spatial points x in [0,1] for plotting

    For each alpha in 'alphas':
      - Build x grid in [0,1]
      - Compute normalized alpha input
      - Get PINN prediction u_pred(x; alpha)
      - Compute true solution u_true(x; alpha)
      - Plot both curves on the same axes.
    """
    plt.figure(figsize=(8, 6))

    # Spatial grid x ∈ [0,1]
    x = torch.linspace(0, 1, num_points, device=device).unsqueeze(1)  # shape (num_points,1)

    for a in alphas:
        # Constant alpha field of shape (num_points,1)
        a_torch = torch.full_like(x, float(a))
        # Normalize to [0,1] for NN input
        a_norm  = normalise_alpha(a_torch)

        # PINN prediction for this alpha
        with torch.no_grad():
            u_pred = model(x, a_norm).cpu().numpy().flatten()

        # Analytical solution (true) evaluated on same x
        u_true = true_solution(x.cpu().numpy(), a).flatten()

        # Plot PINN prediction
        plt.plot(x.cpu().numpy(), u_pred, label=f"PINN α={a}")
        # Plot true solution as dashed line
        plt.plot(x.cpu().numpy(), u_true, linestyle='--')

    plt.title("Parametric PINN Prediction – Constant RHS")
    plt.xlabel("x")
    plt.ylabel("u(x; α)")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()


# Uncomment after training to visualize results.
plot_predictions(pinn)
